# Alternative brain eigenmodes:

This notebook contains the scripts that generate several alternative brain eigenmodes that were used in our evaluations.


##### Package imports

---


In [1]:
import os
import gc
import sys
import glob
import json
import random
import datetime
import importlib
import itertools
import numpy as np
from scipy import spatial
import scipy.sparse as sparse
import scipy.stats as stats
import scipy.io as sio
import pandas as pd
import nibabel as nib
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import seaborn as sns
import boto3
import lapy
import h5py
from sklearn import linear_model

# CSS used for computing local distances and connectome smoothing
from Connectome_Spatial_Smoothing import CSS as css

# Cerebro brain viewer used for visualization
from cerebro import cerebro_brain_utils as cbu
from cerebro import cerebro_brain_viewer as cbv


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


##### Basic functions

---


In [2]:
# Some useful functions

class MyNumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)


def ensure_dir(file_name):
    os.makedirs(os.path.dirname(file_name), exist_ok=True)
    return file_name


def list_dirs(path=os.getcwd()):
    files = glob.glob(os.path.join(path, '*'))
    files = [x for x in files if os.path.isdir(x)]
    return files


def file_exists(file_name, path_name=os.getcwd()):
    return os.path.isfile(os.path.join(path_name, file_name))


def write_json(json_obj, file_path):
    with open(file_path, 'w') as outfile:
        json.dump(json_obj, outfile, sort_keys=True, indent=4,
                  cls=MyNumpyEncoder)
    return json_obj


def load_json(file_path):
    with open(file_path, 'r') as infile:
        return json.load(infile)


def write_np(np_obj, file_path):
    with open(file_path, 'wb') as outfile:
        np.save(outfile, np_obj)


def load_np(file_path):
    with open(file_path, 'rb') as infile:
        return np.load(infile)

# Function to compute density of a high-resolution connectome
def density(adj):
    adj = adj.copy().tocoo()
    return (
        np.sum(adj.row != adj.col) /
        (adj.shape[0] * (adj.shape[0] - 1))
    )


##### Directory structure

---


In [3]:
# path setting
main_dir = os.path.abspath('../')

data_dir = f"{main_dir}/data"


##### Extracting mesh connectivity structure

---


In [4]:
# basic parameters
surface = 'midthickness_MSMAll'

# load an example dscalar
dscalar_file = f'{data_dir}/templates/ones.dscalar.nii'
dscalar = nib.load(dscalar_file)

brain_models = [x for x in dscalar.header.get_index_map(1).brain_models]

# load surfaces for visualization
left_surface_file = f'{data_dir}/templates/S1200.L.{surface}.32k_fs_LR.surf.gii'
left_surface = nib.load(left_surface_file)
right_surface_file = f'{data_dir}/templates/S1200.R.{surface}.32k_fs_LR.surf.gii'
right_surface = nib.load(right_surface_file)

# create a mapping between surface and cifti vertices
left_cortical_surface_model, right_cortical_surface_model = brain_models[0], brain_models[1]
cifti_to_surface = {}
surface_to_cifti = {}
for (i, x) in enumerate(left_cortical_surface_model.vertex_indices):
    cifti_to_surface[i] = x
    surface_to_cifti[x] = i
for (i, x) in enumerate(right_cortical_surface_model.vertex_indices):
    cifti_to_surface[i + right_cortical_surface_model.index_offset] = x + right_surface.darrays[0].data.shape[0]
    surface_to_cifti[x + right_surface.darrays[0].data.shape[0]] = i + right_cortical_surface_model.index_offset

# construct data over surface
surface_mask = list(surface_to_cifti.keys())

left_surface_mask = surface_mask[:right_cortical_surface_model.index_offset]
right_surface_mask = surface_mask[right_cortical_surface_model.index_offset:]


## OSF deposited data:


---

First, downloading data from Pang et al. (2023) that was deposited to [OSF](https://osf.io/xczmp/).


In [ ]:
! mkdir "../data/pang"

Note: If you are planning to replicate our scripts, in the following command, you should replace `<Data_URL>` with a download link acquired from [here](https://osf.io/xczmp/files/osfstorage).

In [ ]:
! curl -JL -o "../data/pang/osf_data.zip" <Data_URL>

Once the data is downloaded, extract the compressed archive:

In [5]:
! ls "../data/pang"

empirical  osf_data.zip  results  template_eigenmodes


In [ ]:
! unzip "../data/pang/osf_data.zip" -d "../data/pang/"

Load the eigenmodes:

In [6]:
### Geometry eigenmodes:
pang_geometric_eigenmodes = np.genfromtxt(f"{data_dir}/pang/results/basis_geometric_midthickness-lh_evec_200.txt")

pang_geometric_eigenmodes.shape


(32492, 200)

In [7]:
### EDR eigenmodes:
with h5py.File(f"{data_dir}/pang/results/basis_connectome_EDR_midthickness-lh_evec_200.mat", 'r') as f:
    pang_edr_eigenmodes = np.array(f['eig_vec']).T

pang_edr_eigenmodes.shape
    

(32492, 200)

In [8]:
### Connectome eigenmodes:
with h5py.File(f"{data_dir}/pang/results/basis_connectome_midthickness-lh_evec_200.mat", 'r') as f:
    pang_connectome_eigenmodes = np.array(f['eig_vec']).T

pang_connectome_eigenmodes.shape
    

(32492, 200)

Store the eigenmodes in a NumPy binary file:

In [9]:
# number of modes
N_modes = 200


In [10]:
write_np(
    pang_geometric_eigenmodes[left_surface_mask, :N_modes],
    ensure_dir(f"{data_dir}/eigenmodes/pang_geometric_eigenmodes.npy")
)


In [11]:
write_np(
    pang_edr_eigenmodes[left_surface_mask, :N_modes],
    ensure_dir(f"{data_dir}/eigenmodes/pang_edr_eigenmodes.npy")
)


In [12]:
write_np(
    pang_connectome_eigenmodes[left_surface_mask, :N_modes],
    ensure_dir(f"{data_dir}/eigenmodes/pang_connectome_eigenmodes.npy")
)
